<a href="https://colab.research.google.com/github/OVP2023/NN/blob/main/%D0%94%D0%97_Resnet_18_%D0%90%D1%80%D1%85%D0%B8%D1%82%D0%B5%D0%BA%D1%82%D1%83%D1%80%D1%8B_%D1%81%D0%B2%D0%B5%D1%80%D1%82%D0%BE%D1%87%D0%BD%D1%8B%D1%85_%D1%81%D0%B5%D1%82%D0%B5%D0%B9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn
import torchvision as tv
from torchsummary import summary
import time
import numpy as np

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

In [3]:
def evaluate_accuracy(data_iter, net):
    acc_sum, n = 0, 0
    net.eval()
    for X, y in data_iter:
        X, y = X.to(device), y.to(device)
        acc_sum += (net(X).argmax(axis=1) == y).sum()
        n += y.shape[0]
    return acc_sum.item() / n

def train(net, train_iter, test_iter, trainer, num_epochs):
    net.to(device)
    loss = nn.CrossEntropyLoss(reduction='sum')
    net.train()
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()

        for i, (X, y) in enumerate(train_iter):
            X, y = X.to(device), y.to(device)
            trainer.zero_grad()
            y_hat = net(X)
            l = loss(y_hat, y)
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.shape[0]

            if i % 10 == 0:
              print(f"Step {i}. time since epoch: {time.time() -  start:.3f}. "
                    f"Train acc: {train_acc_sum / n:.3f}. Train Loss: {train_l_sum / n:.3f}")
        test_acc = evaluate_accuracy(test_iter, net.to(device))
        print('-' * 20)
        print(f'epoch {epoch + 1}, loss {train_l_sum / n:.4f}, train acc {train_acc_sum / n:.3f}'
              f', test acc {test_acc:.3f}, time {time.time() - start:.1f} sec')

In [4]:
transoforms = tv.transforms.Compose([tv.transforms.Grayscale(3), tv.transforms.Resize((224, 224)), tv.transforms.ToTensor()])
Batch_size = 512
train_dataset = tv.datasets.EMNIST('.', split='balanced', train=True, download=True, transform=transoforms)
test_dataset = tv.datasets.EMNIST('.', split='balanced', train=False, download=True, transform=transoforms)

train_data = torch.utils.data.DataLoader(train_dataset, batch_size=Batch_size, shuffle=True)
test_data = torch.utils.data.DataLoader(test_dataset, batch_size=Batch_size)

100%|██████████| 562M/562M [00:04<00:00, 123MB/s]


In [5]:
train_dataset

Dataset EMNIST
    Number of datapoints: 112800
    Root location: .
    Split: Train
    StandardTransform
Transform: Compose(
               Grayscale(num_output_channels=3)
               Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=True)
               ToTensor()
           )

In [6]:
print(len(train_dataset.classes))


47


In [7]:
model_resnet18 = tv.models.resnet18(weights=tv.models.ResNet18_Weights.IMAGENET1K_V1)
for param in model_resnet18.parameters():
    param.requires_grad = False

model_resnet18.fc = torch.nn.Linear(512, 47)

print("Params to learn:")
params_to_update = []
for name, param in model_resnet18.named_parameters():
    if param.requires_grad == True:
        params_to_update.append(param)
        print("\t",name)

optimizer_resnet18 = torch.optim.Adam(params_to_update, lr=0.001)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 65.6MB/s]


Params to learn:
	 fc.weight
	 fc.bias


In [8]:
summary(model_resnet18.to(device), input_size=(3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 112, 112]           9,408
       BatchNorm2d-2         [-1, 64, 112, 112]             128
              ReLU-3         [-1, 64, 112, 112]               0
         MaxPool2d-4           [-1, 64, 56, 56]               0
            Conv2d-5           [-1, 64, 56, 56]          36,864
       BatchNorm2d-6           [-1, 64, 56, 56]             128
              ReLU-7           [-1, 64, 56, 56]               0
            Conv2d-8           [-1, 64, 56, 56]          36,864
       BatchNorm2d-9           [-1, 64, 56, 56]             128
             ReLU-10           [-1, 64, 56, 56]               0
       BasicBlock-11           [-1, 64, 56, 56]               0
           Conv2d-12           [-1, 64, 56, 56]          36,864
      BatchNorm2d-13           [-1, 64, 56, 56]             128
             ReLU-14           [-1, 64,

In [10]:
train(model_resnet18, train_data, test_data, optimizer_resnet18,1)

Step 0. time since epoch: 1.868. Train acc: 0.029. Train Loss: 3.952
Step 10. time since epoch: 15.159. Train acc: 0.114. Train Loss: 3.629
Step 20. time since epoch: 27.783. Train acc: 0.202. Train Loss: 3.370
Step 30. time since epoch: 41.397. Train acc: 0.285. Train Loss: 3.136
Step 40. time since epoch: 53.968. Train acc: 0.344. Train Loss: 2.941
Step 50. time since epoch: 66.987. Train acc: 0.389. Train Loss: 2.776
Step 60. time since epoch: 79.802. Train acc: 0.426. Train Loss: 2.631
Step 70. time since epoch: 92.771. Train acc: 0.454. Train Loss: 2.505
Step 80. time since epoch: 105.626. Train acc: 0.477. Train Loss: 2.396
Step 90. time since epoch: 118.541. Train acc: 0.496. Train Loss: 2.304
Step 100. time since epoch: 131.442. Train acc: 0.512. Train Loss: 2.221
Step 110. time since epoch: 144.355. Train acc: 0.527. Train Loss: 2.146
Step 120. time since epoch: 157.422. Train acc: 0.540. Train Loss: 2.079
Step 130. time since epoch: 170.391. Train acc: 0.553. Train Loss: 2.01